In [1]:
import numpy as np
import cv2
import os
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont
from pathlib import Path

def generate_alphabet_images(alphabet, font_path, font_size, output_dir):
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)
    
    font = ImageFont.truetype(font_path, font_size)
    
    for char in alphabet:
        image = Image.new('L', (100, 100), 'white')
        draw = ImageDraw.Draw(image)
        
        bbox = draw.textbbox((0, 0), char, font=font)
        w, h = bbox[2] - bbox[0], bbox[3] - bbox[1]
        
        draw.text(((100 - w) / 2, (100 - h) / 2), char, 'black', font=font)
        
        bbox = image.getbbox()
        cropped_image = image.crop(bbox)
        cropped_image.save(output_dir / f'{char}.png')

def calculate_features(image_path):
    image_path = Path(image_path)
    if not image_path.exists():
        raise ValueError(f"Image file does not exist: {image_path}")

    try:
        pil_image = Image.open(image_path).convert('L')
        image = np.array(pil_image)
    except Exception as e:
        raise ValueError(f"Failed to load image {image_path} with PIL. Error: {e}")
    
    if image is None:
        raise ValueError(f"Failed to load image {image_path}")
    
    print(f"Image loaded successfully: {image_path}, shape: {image.shape}")
    
    h, w = image.shape
    total_mass = np.sum(image == 0)
    print(f"Total mass: {total_mass}")
    
    # Масса и удельный вес для каждой четверти изображения
    quarters = {
        'Q1': image[:h//2, :w//2],
        'Q2': image[:h//2, w//2:],
        'Q3': image[h//2:, :w//2],
        'Q4': image[h//2:, w//2:]
    }
    
    features = {}
    
    for key, quarter in quarters.items():
        mass = np.sum(quarter == 0)
        print(f"{key} mass: {mass}")
        features[f'{key}_mass'] = mass
        features[f'{key}_mass_normalized'] = mass / (quarter.size if quarter.size > 0 else 1)
    
    # Моменты инерции и центр тяжести
    binary_image = (image == 0).astype(np.uint8)  # Преобразование в тип uint8
    try:
        M = cv2.moments(binary_image)
        print(f"Moments: {M}")
    except Exception as e:
        raise ValueError(f"Error computing moments for {image_path}: {e}")
    
    cX = int(M["m10"] / M["m00"]) if M["m00"] != 0 else 0
    cY = int(M["m01"] / M["m00"]) if M["m00"] != 0 else 0
    features['center_of_mass_x'] = cX
    features['center_of_mass_y'] = cY
    
    # Нормированные координаты центра тяжести
    normalized_cX = cX / w if w > 0 else 0
    normalized_cY = cY / h if h > 0 else 0
    features['normalized_center_of_mass_x'] = normalized_cX
    features['normalized_center_of_mass_y'] = normalized_cY
    
    # Осевые моменты инерции
    mu20 = M["mu20"] / M["m00"] if M["m00"] != 0 else 0
    mu02 = M["mu02"] / M["m00"] if M["m00"] != 0 else 0
    features['inertia_horizontal'] = mu20
    features['inertia_vertical'] = mu02
    
    # Нормированные осевые моменты инерции
    normalized_mu20 = mu20 / (w * w + h * h) if (w * w + h * h) > 0 else 0
    normalized_mu02 = mu02 / (w * w + h * h) if (w * w + h * h) > 0 else 0
    features['normalized_inertia_horizontal'] = normalized_mu20
    features['normalized_inertia_vertical'] = normalized_mu02
    
    # Профили X и Y
    profile_x = np.sum(image == 0, axis=0)
    profile_y = np.sum(image == 0, axis=1)
    
    output_dir = Path('profiles')
    output_dir.mkdir(parents=True, exist_ok=True)
    np.savetxt(output_dir / f'{image_path.stem}_profile_x.txt', profile_x, fmt='%d')
    np.savetxt(output_dir / f'{image_path.stem}_profile_y.txt', profile_y, fmt='%d')
    
    return features

def plot_profiles(image_path, output_dir):
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)
    
    image = cv2.imread(str(image_path), cv2.IMREAD_GRAYSCALE)
    if image is None:
        raise ValueError(f"Failed to load image {image_path}")
    
    x_profile = np.sum(image == 0, axis=0)
    y_profile = np.sum(image == 0, axis=1)
    
    char = os.path.splitext(os.path.basename(image_path))[0]
    
    plt.figure(figsize=(10, 4))
    
    plt.subplot(1, 2, 1)
    plt.bar(range(len(x_profile)), x_profile)
    plt.title(f'X Profile of {char}')
    plt.xlabel('X')
    plt.ylabel('Sum of Black Pixels')
    
    plt.subplot(1, 2, 2)
    plt.bar(range(len(y_profile)), y_profile)
    plt.title(f'Y Profile of {char}')
    plt.xlabel('Y')
    plt.ylabel('Sum of Black Pixels')
    
    plt.tight_layout()
    plt.savefig(output_dir / f'{char}_profile.png')
    plt.close()
    print(f"Profiles saved for {char}")

if __name__ == '__main__':
    alphabet = '1' 
    # ABCDEFGHIJKLMNOPQRSTUVWXYZÑ'
    font_path = 'times_new_roman1.ttf'
    font_size = 52
    output_dir = 'alphabet_images'
    profile_output_dir = 'profiles'


    # generate_alphabet_images(alphabet, font_path, font_size, output_dir)

    for image_file in Path(output_dir).iterdir():
        print(f"Found image file: {image_file}")

    output_features = []

    for char in alphabet:
        image_path = Path(output_dir) / f'{char}.png'
    
        try:
            print(f"Processing image: {image_path}")
            features = calculate_features(image_path)
            print(f"Features calculated for {char}: {features}")
            features['character'] = char
            output_features.append(features)
            plot_profiles(image_path, profile_output_dir)
            print(f"Profiles plotted for {char}")
        except (FileNotFoundError, ValueError) as e:
            print(e)

    try:
        df = pd.DataFrame(output_features)
        print(f"DataFrame created with {len(df)} entries")
        df.to_csv('features.csv', index=False, sep=';')
        print("Features saved to 'features.csv'")
    except Exception as e:
        print(f"Error creating DataFrame or saving CSV: {e}")

 


Found image file: alphabet_images\.ipynb_checkpoints
Found image file: alphabet_images\1.png
Found image file: alphabet_images\A.PNG
Found image file: alphabet_images\B.PNG
Found image file: alphabet_images\C.PNG
Found image file: alphabet_images\D.PNG
Found image file: alphabet_images\E.PNG
Found image file: alphabet_images\F.PNG
Found image file: alphabet_images\G.PNG
Found image file: alphabet_images\H.PNG
Found image file: alphabet_images\I.PNG
Found image file: alphabet_images\J.PNG
Found image file: alphabet_images\K.PNG
Found image file: alphabet_images\L.PNG
Found image file: alphabet_images\M.PNG
Found image file: alphabet_images\N.PNG
Found image file: alphabet_images\O.PNG
Found image file: alphabet_images\P.PNG
Found image file: alphabet_images\Q.PNG
Found image file: alphabet_images\R.PNG
Found image file: alphabet_images\S.PNG
Found image file: alphabet_images\T.PNG
Found image file: alphabet_images\U.PNG
Found image file: alphabet_images\V.PNG
Found image file: alphabet_